In [4]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
import re
import pandas as pd 
import glob
import scipy
import scipy.io
from collections import Counter
import scanpy as sc
import gseapy as gp
import h5py
import louvain
import harmonypy as hm
import scrublet as scr
from matplotlib import rcParams
rcParams['figure.figsize'] = 12,12
file_figs = ''
from methods import *
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)
sc.settings.figdir = '../figures/ec_subtypes/'

The `sinfo` package has changed name and is now called `session_info` to become more discoverable and self-explanatory. The `sinfo` PyPI package will be kept around to avoid breaking old installs and you can downgrade to 0.3.2 if you want to use it without seeing this message. For the latest features and bug fixes, please install `session_info` instead. The usage and defaults also changed slightly, so please review the latest README at https://gitlab.com/joelostblom/session_info.
-----
anndata     0.7.6
scanpy      1.7.2
sinfo       0.3.4
-----
PIL                 8.1.2
anyio               NA
appdirs             1.4.4
argon2              20.1.0
attr                21.2.0
babel               2.9.1
backcall            0.2.0
bioservices         1.6.0
brotli              NA
bs4                 4.9.3
certifi             2020.12.05
cffi                1.14.5
chardet             4.0.0
colorama            0.4.4
colorlog            NA
cycler              0.10.0
cython_runtime      NA
dateutil  

In [3]:
gtex_score = pd.read_csv('gtex/heart.links.score.hg38.tsv', sep = '\t')
gtex_overlap = pd.read_csv('gtex/heart.links.pks.ovlpWithGWAS.hg38.tsv',  sep = '\t')

In [4]:
gtex_overlap

,imputed_CARDIoGRAM_C4D_CAD_ADDITIVE,imputed_CNCR_Insomnia_all,imputed_DIAGRAM_T2D_TRANS_ETHNIC,imputed_GABRIEL_Asthma,imputed_GPC-NEO-NEUROTICISM,imputed_HRGene_HeartRate,imputed_IBD.EUR.Crohns_Disease,imputed_IBD.EUR.Inflammatory_Bowel_Disease,imputed_IBD.EUR.Ulcerative_Colitis,imputed_IGAP_Alzheimer,imputed_PGC_ADHD_EUR_2017,imputed_PGC_ASD_2017_CEU,imputed_TAGC_Asthma_EUR,imputed_UKB_20002_1261_self_reported_multiple_sclerosis,imputed_UKB_20002_1262_self_reported_parkinsons_disease,imputed_UKB_20002_1289_self_reported_schizophrenia,imputed_UKB_20127_Neuroticism_score,imputed_UKB_6152_8_diagnosed_by_doctor_Asthma
chr10:100042943-100046361,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
chr10:100327097-100327617,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
chr10:100362760-100363304,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
chr10:100882358-100883570,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
chr10:100911755-100914215,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr9:98192793-98193611,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
chr9:98888547-98888909,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
chr9:98893923-98894887,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
chr9:98894943-98895889,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
gtex_score['region_id'] = [x.split(';')[1] for x in gtex_score.index]

In [19]:
##First : select the links with overlap with CAD
print(gtex_overlap.columns)
overlapp_CAD_id = gtex_overlap[gtex_overlap.imputed_CARDIoGRAM_C4D_CAD_ADDITIVE == 1].index

Index(['imputed_CARDIoGRAM_C4D_CAD_ADDITIVE', 'imputed_CNCR_Insomnia_all',
       'imputed_DIAGRAM_T2D_TRANS_ETHNIC', 'imputed_GABRIEL_Asthma',
       'imputed_GPC-NEO-NEUROTICISM', 'imputed_HRGene_HeartRate',
       'imputed_IBD.EUR.Crohns_Disease',
       'imputed_IBD.EUR.Inflammatory_Bowel_Disease',
       'imputed_IBD.EUR.Ulcerative_Colitis', 'imputed_IGAP_Alzheimer',
       'imputed_PGC_ADHD_EUR_2017', 'imputed_PGC_ASD_2017_CEU',
       'imputed_TAGC_Asthma_EUR',
       'imputed_UKB_20002_1261_self_reported_multiple_sclerosis',
       'imputed_UKB_20002_1262_self_reported_parkinsons_disease',
       'imputed_UKB_20002_1289_self_reported_schizophrenia',
       'imputed_UKB_20127_Neuroticism_score',
       'imputed_UKB_6152_8_diagnosed_by_doctor_Asthma'],
      dtype='object')


In [20]:
len(overlapp_CAD_id)

17

In [21]:
#gtex_score_sel = gtex_score[gtex_score.region_id.isin(overlapp_CAD_id)]
gtex_score_sel = gtex_score[(gtex_score["coloc.H4.ABF"] >= 0.25) | (gtex_score["MR.log10pval"] >= 1.31) |
                             (gtex_score["PGS.pcor.log10pval"] >= 1.31)]

In [22]:
gtex_score_sel['gene_id'] = [x.split(';')[0] for x in gtex_score_sel.index]
gtex_score_sel['gene_id_2'] =  [x.split('.')[0] for x in gtex_score_sel.gene_id]
gtex_score_sel = gtex_score_sel[gtex_score_sel['gene_id_2'].isin(adata.raw.var.gene_ids)]
gtex_score_sel['chr'] = [x.split(':')[0] for x in gtex_score_sel.region_id]
gtex_score_sel['start_end'] = [x.split(':')[1] for x in gtex_score_sel.region_id]
gtex_score_sel['start'] = [x.split('-')[0] for x in gtex_score_sel.start_end]
gtex_score_sel['end'] = [x.split('-')[1] for x in gtex_score_sel.start_end]
gene_names = []
for gene in gtex_score_sel.gene_id_2:
    gene_names.append(adata.raw.var[adata.raw.var.gene_ids == gene].index.to_list()[0])
gtex_score_sel['gene_names'] = gene_names

<ipython-input-22-c525fadc2abf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lei_score_sel['gene_id'] = [x.split(';')[0] for x in lei_score_sel.index]
<ipython-input-22-c525fadc2abf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lei_score_sel['gene_id_2'] =  [x.split('.')[0] for x in lei_score_sel.gene_id]


In [23]:
gtex_score_sel

,coloc.H4.ABF,MR.log10pval,PGS.pcor.log10pval,FMeQTL.distInv,distance.inverse,region_id,gene_id,gene_id_2,chr,start_end,start,end,gene_names
ENSG00000000938.12;chr1:27475401-27477136,NaN,NaN,1.358734,0.0,0.000006,chr1:27475401-27477136,ENSG00000000938.12,ENSG00000000938,chr1,27475401-27477136,27475401,27477136,FGR
ENSG00000000938.12;chr1:27496953-27497691,NaN,1.407380,0.817937,0.0,0.000007,chr1:27496953-27497691,ENSG00000000938.12,ENSG00000000938,chr1,27496953-27497691,27496953,27497691,FGR
ENSG00000000938.12;chr1:27562998-27565691,0.157509,2.343468,0.511364,0.0,0.000014,chr1:27562998-27565691,ENSG00000000938.12,ENSG00000000938,chr1,27562998-27565691,27562998,27565691,FGR
ENSG00000000938.12;chr1:27755570-27756246,NaN,NaN,1.393687,0.0,0.000008,chr1:27755570-27756246,ENSG00000000938.12,ENSG00000000938,chr1,27755570-27756246,27755570,27756246,FGR
ENSG00000000938.12;chr1:27872521-27875693,NaN,NaN,1.573608,0.0,0.000004,chr1:27872521-27875693,ENSG00000000938.12,ENSG00000000938,chr1,27872521-27875693,27872521,27875693,FGR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283633.1;chr22:16601891-16602279,NaN,NaN,2.087941,0.0,0.000095,chr22:16601891-16602279,ENSG00000283633.1,ENSG00000283633,chr22,16601891-16602279,16601891,16602279,AP000547.3
ENSG00000283633.1;chr22:16602481-16603268,NaN,NaN,1.887857,0.0,0.000103,chr22:16602481-16603268,ENSG00000283633.1,ENSG00000283633,chr22,16602481-16603268,16602481,16603268,AP000547.3
ENSG00000283646.1;chr3:46405359-46405829,NaN,3.415034,0.003183,0.0,0.000056,chr3:46405359-46405829,ENSG00000283646.1,ENSG00000283646,chr3,46405359-46405829,46405359,46405829,LINC02009
ENSG00000284237.1;chr1:207893244-207893594,NaN,0.787810,1.954242,0.0,0.000015,chr1:207893244-207893594,ENSG00000284237.1,ENSG00000284237,chr1,207893244-207893594,207893244,207893594,AL356275.1


In [24]:
df_snp_gtex = pd.read_csv('imputed_CARDIoGRAM_C4D_CAD_ADDITIVE.bed', sep = '\t')

In [25]:
df_snp_gtex = df_snp_gtex[df_snp_gtex.pval < 0.000001]

In [26]:
df_snp_gtex

,#chr,start,end,a1,a2,zscore,pval
120420,chr1,37921054,37921055,G,A,-4.8948,9.840000e-07
167484,chr1,55021974,55021975,A,G,5.3179,1.050000e-07
167503,chr1,55023868,55023869,C,T,5.2563,1.470000e-07
167544,chr1,55028343,55028344,T,C,5.3315,9.740000e-08
167559,chr1,55030365,55030366,T,C,5.5848,2.340000e-08
...,...,...,...,...,...,...,...
9315637,chr9,133278536,133278537,C,T,-4.9421,7.730000e-07
9315638,chr9,133278723,133278724,T,C,-6.4471,1.140000e-10
9315645,chr9,133279293,133279294,G,T,-6.4283,1.290000e-10
9315646,chr9,133279426,133279427,C,T,-6.5876,4.470000e-11


In [27]:
df_snp_gtex['CHR'] = df_snp_gtex['#chr']
df_snp_gtex['POS'] = df_snp_gtex['start']

In [28]:
SNP_inside_gtex = [True in ((df_snp_gtex['#chr'].isin([gtex_score_sel.loc[lnkid, 'chr']])) 
                       & (df_snp_gtex.end <= int(gtex_score_sel.loc[lnkid, 'end'])) 
                       & (df_snp_gtex.start >= int(gtex_score_sel.loc[lnkid, 'start']))).to_list() for lnkid in gtex_score_sel.index]

In [29]:
True in SNP_inside_gtex

True

In [30]:
SNP_inside_perso = [True in ((pd.Series(['chr' + str(x) for x in df_snps_all.CHR], index = df_snps_all.index).isin([gtex_score_sel.loc[lnkid, 'chr']])) 
                       & (df_snps_all.POS <= int(gtex_score_sel.loc[lnkid, 'end'])) 
                       & (df_snps_all.POS >= int(gtex_score_sel.loc[lnkid, 'start']))).to_list() for lnkid in gtex_score_sel.index]

In [31]:
True in SNP_inside_perso

True

In [32]:
gtex_score_non_sel = gtex_score_sel.copy()

In [33]:
gtex_score_sel = gtex_score_sel[SNP_inside_gtex]

In [34]:
gtex_score_sel

,coloc.H4.ABF,MR.log10pval,PGS.pcor.log10pval,FMeQTL.distInv,distance.inverse,region_id,gene_id,gene_id_2,chr,start_end,start,end,gene_names
ENSG00000041357.15;chr15:78803812-78805417,0.183710,3.063299,2.421408,0.000000,0.000004,chr15:78803812-78805417,ENSG00000041357.15,ENSG00000041357,chr15,78803812-78805417,78803812,78805417,PSMA4
ENSG00000084092.6;chr4:56907958-56908743,NaN,NaN,1.472941,0.000000,0.000014,chr4:56907958-56908743,ENSG00000084092.6,ENSG00000084092,chr4,56907958-56908743,56907958,56908743,NOA1
ENSG00000084674.14;chr2:21041757-21043057,0.991092,6.302453,6.661623,0.001953,0.000600,chr2:21041757-21043057,ENSG00000084674.14,ENSG00000084674,chr2,21041757-21043057,21041757,21043057,APOB
ENSG00000100024.14;chr22:24348260-24349653,0.225472,3.848682,0.639021,0.000000,0.000007,chr22:24348260-24349653,ENSG00000100024.14,ENSG00000100024,chr22,24348260-24349653,24348260,24349653,UPB1
ENSG00000103811.15;chr15:78750123-78751620,0.021383,1.758398,0.045028,0.000000,0.000005,chr15:78750123-78751620,ENSG00000103811.15,ENSG00000103811,chr15,78750123-78751620,78750123,78751620,CTSH
ENSG00000103811.15;chr15:78796885-78799054,0.029874,1.668368,0.635937,0.000000,0.000007,chr15:78796885-78799054,ENSG00000103811.15,ENSG00000103811,chr15,78796885-78799054,78796885,78799054,CTSH
ENSG00000103811.15;chr15:78803812-78805417,0.241923,1.109707,1.862911,0.000000,0.000007,chr15:78803812-78805417,ENSG00000103811.15,ENSG00000103811,chr15,78803812-78805417,78803812,78805417,CTSH
ENSG00000115486.11;chr2:85539132-85540756,NaN,0.252374,3.543777,0.000000,0.000046,chr2:85539132-85540756,ENSG00000115486.11,ENSG00000115486,chr2,85539132-85540756,85539132,85540756,GGCX
ENSG00000136378.14;chr15:78750123-78751620,0.547950,0.895954,0.617831,0.000000,0.000017,chr15:78750123-78751620,ENSG00000136378.14,ENSG00000136378,chr15,78750123-78751620,78750123,78751620,ADAMTS7
ENSG00000136378.14;chr15:78789538-78791285,0.306620,0.489548,1.495713,0.000000,0.000048,chr15:78789538-78791285,ENSG00000136378.14,ENSG00000136378,chr15,78789538-78791285,78789538,78791285,ADAMTS7


In [35]:
linked_genes_list = list(set(gtex_score_sel.gene_names))

# epimap Links

In [7]:
links_files = []
links_files.append(pd.read_csv('epimap/21378988.tsv', sep = '\t'))
links_files.append(pd.read_csv('epimap/21378990.tsv', sep = '\t'))
links_files.append(pd.read_csv('epimap/24262325-2.tsv', sep = '\t'))
links_files.append(pd.read_csv('epimap/24262325-3.tsv', sep = '\t'))
links_files.append(pd.read_csv('epimap/24262325.tsv', sep = '\t'))
links_files.append(pd.read_csv('epimap/26343387.tsv', sep = '\t'))
links_files.append(pd.read_csv('epimap/28714975.tsv', sep = '\t'))
links_files.append(pd.read_csv('epimap/29212778.tsv', sep = '\t'))
epimap_links = pd.concat(links_files)
print(epimap_links.shape)
epimap_links = epimap_links[epimap_links.linkedGene.isin(adata.raw.var.index)]
print(epimap_links.shape)

(18820, 12)
(14070, 12)


In [8]:
epimap_links.index = range(epimap_links.shape[0])

In [9]:
snps_epimap = pd.read_csv('epimap/gwascatalog_may03_2019_noquotes_rsid.txt', sep = '\t')

In [10]:
snps_epimap['chrom_pos'] = ['chr' + str(snps_epimap.loc[x, 'chrom']) + '-' + str(snps_epimap.loc[x, 'chromStart']) for x in snps_epimap.index]

In [11]:
epimap_links['chrom_pos'] = [epimap_links.loc[x, 'chr'] + '-' + str(epimap_links.loc[x, 'snpPos']) for x in epimap_links.index]

In [12]:
epimap_links

,chr,snpPos,snp.pValue,distToCenter,nearestGene,linkedGene,linkScore,linkDist,enrRank,enrName,enr.pValue,enrGroup,chrom_pos
0,chr9,22098574,0.000000,744.5,CDKN2B,CDKN2A,0.92,102529.0,1,Bronchial Epithelial Cell,0.000000e+00,Epithelial,chr9-22098574
1,chr9,22098574,0.000000,744.5,CDKN2B,CDKN2B,0.90,88525.0,1,Bronchial Epithelial Cell,0.000000e+00,Epithelial,chr9-22098574
2,chr9,22098574,0.000000,634.5,CDKN2B,CDKN2B,0.88,88635.0,1,Bronchial Epithelial Cell,0.000000e+00,Epithelial,chr9-22098574
3,chr9,22098574,0.000000,634.5,CDKN2B,CDKN2A,0.86,102639.0,1,Bronchial Epithelial Cell,0.000000e+00,Epithelial,chr9-22098574
4,chr9,22098574,0.000000,744.5,CDKN2B,CDKN2B,0.94,88525.0,2,Trophoblast Deriv,1.000000e-16,ES-deriv,chr9-22098574
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14065,chr12,107752077,0.000009,1150.5,BTBD11,BTBD11,0.93,41029.0,25,Olfactory Neurosphere,0.000000e+00,Neurosph,chr12-107752077
14066,chr12,107752077,0.000009,1526.0,BTBD11,BTBD11,0.92,38352.5,25,Olfactory Neurosphere,0.000000e+00,Neurosph,chr12-107752077
14067,chr17,59472057,0.000009,727.5,TBX2,TBX2,0.91,-5875.0,25,Olfactory Neurosphere,0.000000e+00,Neurosph,chr17-59472057
14068,chr17,59472057,0.000009,576.0,TBX2,TBX4,0.89,-57132.5,25,Olfactory Neurosphere,0.000000e+00,Neurosph,chr17-59472057


In [13]:
snps_epimap = snps_epimap[snps_epimap.chrom_pos.isin(epimap_links.chrom_pos)]

In [15]:
snps_epimap.uid.value_counts()

29212778 - Coronary artery disease                        317
29507422 - Low density lipoprotein cholesterol levels      47
29507422 - Total cholesterol levels                        44
30072576 - Blood protein levels                            43
29507422 - High density lipoprotein cholesterol levels     29
                                                         ... 
26192919 - Inflammatory bowel disease                       1
19060911 - LDL cholesterol                                  1
26752265 - Glaucoma (primary open-angle)                    1
26390057 - Pulse pressure                                   1
28017375 - Hemoglobin levels                                1
Name: uid, Length: 384, dtype: int64

In [16]:
epimap_links = epimap_links[epimap_links.chrom_pos.isin(snps_epimap.chrom_pos)]

In [17]:
epimap_links.shape

(14070, 13)

In [19]:
len(list(set(epimap_links.linkedGene)))

759

In [61]:
linked_genes_list = linked_genes_list + list(set(epimap_links.linkedGene))

In [62]:
len(linked_genes_list)

774

In [64]:
#linked_genes_list

In [68]:
genes_list_df = pd.DataFrame(linked_genes_list, columns = ["genes"])

In [69]:
genes_list_df

,genes
0,CTSH
1,MFSD13A
2,CHRNA5
3,NOA1
4,ADAMTS7
...,...
769,HERPUD1
770,C18orf32
771,ZC3H12A
772,SRR


In [70]:
genes_list_df.to_csv("genes_list.csv", sep = "\t")